In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import pathlib 
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

import PT_files.save_load as sl
from DnCNN_NP.layers  import relu, np_BatchNorm2d

import time 
from collections import OrderedDict
import pdb

**The goal of this notebook is to see how we can optimize the code for `np.Conv2d`.** 

**`get_indices` work:**

We know we can increase the speed of `get_indices` because we just have to call it three times and then save the results and just load them in when needed. That way it doesn't have to be run every single time and take up ~4 seconds (when in the middle of the model).

- This is possible because `get_indices` is just getting the correct indices for the 2d array indexing used to convert our multi-dimensional image and weight objects into large 2d arrays to allow for matrix matrix multiplication.

- We have to call it three times for the 3 different shape transformations that take place within the model ie.
    1. Transforming input image data into multiple filters ie. 1 channel -> 64 channels
    2. Transforming input data into same size ie. 64 channels -> 64 channels
    3. Transforming input data into original size ie. 64 channels -> 1 channel
- Doing it this way we save ~ 75 seconds ie. 8.5X faster than current implementation of calling `get_indices` every time we call `np.Conv2d`

**`im2col` work:**

We know the bottleneck in `im2col` is from the array slicing we do to convert our image to columns. We also know the next line with the `np.concatenate` is fairly fast (~ 1 second)

- The bottleneck in the array slicing is potentially due to the hardware running into latency-bound computations
    - Meaning due to the conversion of images to columns the data is no longer row-major (the default way data is stored in Python) and thus there must be more calls by the GPU to fetch the requested data
    - This could be fixed by transposing the data array (ie. the data is now back into the hardware friendly row-major form) and this would allow for the GPU to fetch the requested data as well as take advantage of the cache-ing that hardware does (ie. tries to antipicate what data will be loaded next, by not just loading the data you requested, but loading in the left/right neighbors of the data requested)
    - [This](https://stackoverflow.com/questions/67826377/efficient-numpy-slicing-of-a-large-3d-array) stack overeflow page explains it better with some example code
    
- Another area to explore would be instead of doing normal array slicing using `np.take`. 
    - Haven't looked at it thoroughly, but from [this](https://stackoverflow.com/questions/62205777/speed-up-numpy-indexing-of-large-array) stackoverflow there does seem to be a speed-up

In [2]:
PATH = pathlib.Path(os.getenv('PSCRATCH'))
DATA = PATH / 'DESI_dn' /'Model_params'
assert DATA.exists()
# name = '6k_model_wb_e800_lys20_58feat.pth'
name = '2k_model_bs64_e800_ps50_Adam.pth'
# weights = np.load(DATA / name)
weights = torch.load(str(DATA / name))


#Load the actual data that we're working on & print the shape of this data
test_data = sl.NERSC_load('test_data_40%_6000.npy')
sample = test_data[0]
print('Shape of test set=', sample.shape)

samp = sample[0][0][1000:3000, 1000:3000]
samp = samp.reshape((1, 1, 2000, 2000))

Shape of test set= (108, 1, 6000, 6000)


In [3]:
def get_indices(input_data, weights_dict, prefix, stride=1, padding=1):
    get_indices_start = time.perf_counter()

    # Get input size
    
    # Checking to see if a single sample or a batch of samples is given.
    # If batch take the batch_size, in_channels, H, and W
    # If single sample is given reshape so the values above can be calculated
    if len(input_data.shape) == 4:
    
        batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
        
    elif len(input_data.shape) == 3:
        
        input_data = input_data.reshape((1, 1, 2000 , 2000))
        batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
        
    # Load the weights and biases needed for a convolution
    # then take off gpu memory, move to CPU memory,
    # and lastly transform to numpy
    weight = weights_dict[str(prefix) + 'weight']
    weight = weight.detach().cpu().numpy()
    
    bias = weights_dict[str(prefix) + 'bias']
    bias = bias.detach().cpu().numpy()
    
    # Calculate the kernel size and output channels from
    # the loaded weights from above
    kernel_size = weight[0][0].shape
    output_channels = len(weight)
    
    # Calculations for the output H and W dimensions.
    height_out = ((height + (2*padding) - (kernel_size[0] - 1) - 1) / stride) + 1
    height_out = int(height_out)
    width_out = ((width + (2*padding) - (kernel_size[1] - 1) - 1) / stride) + 1
    width_out = int(width_out)
    
    
    # ----Compute matrix of index i----

    # Level 1 vector.
    level1 = np.repeat(np.arange(kernel_size[0]), kernel_size[1])
    # Duplicate for the other channels.
    level1 = np.tile(level1, input_channels)
    # Create a vector with an increase by 1 at each level.
    everyLevels = stride * np.repeat(np.arange(height_out), width_out)
    # Create matrix of index i at every levels for each channel.
    i = level1.reshape(-1, 1) + everyLevels.reshape(1, -1)
    
    # ----Compute matrix of index j----
    
    # Slide 1 vector.
    slide1 = np.tile(np.arange(kernel_size[1]), kernel_size[0])
    # Duplicate for the other channels.
    slide1 = np.tile(slide1, input_channels)
    # Create a vector with an increase by 1 at each slide.
    everySlides = stride * np.tile(np.arange(width_out), height_out)
    # Create matrix of index j at every slides for each channel.
    j = slide1.reshape(-1, 1) + everySlides.reshape(1, -1)
    
    # ----Compute matrix of index d----

    # This is to mark delimitation for each channel
    # during multi-dimensional arrays indexing.
    d = np.repeat(np.arange(input_channels), kernel_size[0] * kernel_size[1]).reshape(-1, 1)
    
    get_indices_end = time.perf_counter()
    print('get_indices takes:', get_indices_end-get_indices_start, 'seconds')
    
    return i, j, d

def im2col(input_data, weights_dict, prefix, stride=1, padding=1):
    """
        Transforms our input image into a matrix.

        Parameters:
        -----------
        input_data: nd.array
            The input image(s)
        weights_dict: OrderedDict
            Dictionary containing the PyTorch trained weights for every 
            layer of the model
        prefix: str
            The prefix that picks out the specific layer's weights to be used
            E.g. prefix='layers.0.0.' would be the first layers convolutional
            weights and bias's

        Returns:
        --------
        cols: output matrix.
    """
    im2col_start = time.perf_counter()

    if len(input_data.shape) == 4:
    
        batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
        
    elif len(input_data.shape) == 3:
        
        input_data = input_data.reshape((1, 1, 2000 , 2000))
        batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)

    # Padding
    input_padded = np.pad(input_data, ((0,0), (0,0), (padding, padding), (padding, padding)), mode='constant')
    i, j, d = get_indices(input_data=input_data, weights_dict=weights_dict, prefix=prefix)
    # Multi-dimensional arrays indexing.
    cols = input_padded[:, d, i, j]
    cols = np.concatenate(cols, axis=-1)
    
    im2col_end = time.perf_counter()
    print('Im2col takes:', im2col_end-im2col_start, 'seconds')
    
    return cols

def np_Conv2d(input_data, weights_dict, prefix):
    """
        Performs a forward convolution.

        Parameters:
        - X : Last conv layer of shape (m, n_C_prev, n_H_prev, n_W_prev).
        Returns:
        - out: previous layer convolved.
    """
    
    conv_start = time.perf_counter()
    if len(input_data.shape) == 4:
    
        batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
        
    elif len(input_data.shape) == 3:
        
        input_data = input_data.reshape((1, 1, 2000 , 2000))
        batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)


    output_channels = len(weights_dict[str(prefix) + 'weight']) # num_of_filters
    height_out = int((height + 2 * 1 - 3)/ 1) + 1
    width_out = int((width + 2 * 1 - 3)/ 1) + 1

    X_col = im2col(input_data=input_data, weights_dict=weights_dict, prefix=prefix)
    w_col = weights_dict[str(prefix) + 'weight'].detach().cpu().numpy().reshape((output_channels, -1))
    b_col = weights_dict[str(prefix) + 'bias'].detach().cpu().numpy().reshape(-1, 1)
    # Perform matrix multiplication.
    out = w_col @ X_col + b_col
    # Reshape back matrix to image.
    out = np.array(np.hsplit(out, batch_size)).reshape((batch_size, output_channels, height_out, width_out))
    
    conv_end = time.perf_counter()
    print('Conv takes:', conv_end-conv_start, 'seconds')
    return out

In [4]:
# First layer convolution

# Note: The time for im2col is the time for im2col as well as get_indices
# due to im2col calling get_indices

# Note: This is the same for conv, except it calls im2col, which then calls
# get_indices, thus you need to subtract the time of the previous function
# against the current function to get the correct time
i, j, d = get_indices(input_data=samp, weights_dict=weights, prefix='layers.0.0.');
print('i shape =', i.shape)
print('j shape =',j.shape)
print('d shape =',d.shape)
print()
im2col(input_data=samp, weights_dict=weights, prefix='layers.0.0.');
print()
out = np_Conv2d(input_data=samp, weights_dict=weights, prefix='layers.0.0.');

get_indices takes: 0.09291511698393151 seconds
i shape = (9, 4000000)
j shape = (9, 4000000)
d shape = (9, 1)

get_indices takes: 0.09246833799988963 seconds
Im2col takes: 0.27658326100208797 seconds

get_indices takes: 0.09049165199394338 seconds
Im2col takes: 0.26960439400863834 seconds
Conv takes: 0.5580244529992342 seconds


In [5]:
# Second layer convolution
get_indices(input_data=out, weights_dict=weights, prefix='layers.1.0.');
print()
im2col(input_data=out, weights_dict=weights, prefix='layers.1.0.');
print()
out2 = np_Conv2d(input_data=out, weights_dict=weights, prefix='layers.1.0.');

get_indices takes: 4.693190552003216 seconds

get_indices takes: 4.649339191004401 seconds
Im2col takes: 16.331543762004003 seconds

get_indices takes: 4.535428042989224 seconds
Im2col takes: 16.293198670988204 seconds
Conv takes: 18.492726391006727 seconds


# 0. Experimenting on how to optimize the bottleneck found in im2col

In [6]:
# def im2col_testing(input_data, weights_dict, prefix, stride=1, padding=1):
#     """
#         Transforms our input image into a matrix.

#         Parameters:
#         - X: input image.
#         - HF: filter height.
#         - WF: filter width.
#         - stride: stride value.
#         - pad: padding value.

#         Returns:
#         -cols: output matrix.
#     """
#     im2col_start = time.perf_counter()

#     if len(input_data.shape) == 4:
    
#         batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
        
#     elif len(input_data.shape) == 3:
        
#         input_data = input_data.reshape((1, 1, 2000 , 2000))
#         batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
    
#     # Padding
#     input_padded = np.pad(input_data, ((0,0), (0,0), (padding, padding), (padding, padding)), mode='constant')
#     i, j, d = get_indices(input_data=input_data, weights_dict=weights_dict, prefix=prefix)
    
#     array_indexing_start = time.perf_counter()
#     # Multi-dimensional arrays indexing.
#     array_slicing_start = time.perf_counter()
#     iT, jT, dT = i.T, j.T, d.T
#     transposed_input_padded = input_padded.T
#     cols = transposed_input_padded[:, iT, jT, dT].T
#     array_slicing_end = time.perf_counter()
#     print('Im2col array slicing takes:', array_slicing_end-array_slicing_start, 'seconds')
#     print()    
#     cols = np.concatenate(cols, axis=-1)
    
#     im2col_end = time.perf_counter()
#     print('Im2col takes:', im2col_end-im2col_start, 'seconds')
    
#     return cols

In [7]:
input_data=out
weights_dict = weights
prefix = 'layers.1.0.'
padding=1
stride=1

im2col_start = time.perf_counter()
if len(input_data.shape) == 4:
    
    batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
        
elif len(input_data.shape) == 3:

    input_data = input_data.reshape((1, 1, 2000 , 2000))
    batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)

# Padding
input_padded = np.pad(input_data, ((0,0), (0,0), (padding, padding), (padding, padding)), mode='constant')
i, j, d = get_indices(input_data=input_data, weights_dict=weights_dict, prefix=prefix)
# Multi-dimensional arrays indexing.
cols = input_padded[:, d, i, j]
cols = np.concatenate(cols, axis=-1)

im2col_end = time.perf_counter()
print('Total Im2col takes:', im2col_end-im2col_start, 'seconds')

get_indices takes: 4.696664950984996 seconds
Total Im2col takes: 16.13210764498217 seconds


In [9]:
idx = np.ravel_multi_index(([0], d, i, j), input_padded.shape)

In [10]:
idx.shape

(576, 4000000)

In [11]:
cols2 = input_padded.reshape(-1)[idx]

In [14]:
np.allclose(cols2, cols)

True

In [8]:
print('i shape =', i.shape)
print('j shape =',j.shape)
print('d shape =',d.shape)
print('input_padded shape =',input_padded.shape)
print('cols shape before concatenation =', input_padded[:, d, i, j].shape)
print('cols shape =', cols.shape) # cols = np.concatenate(cols, axis=-1)

i shape = (576, 4000000)
j shape = (576, 4000000)
d shape = (576, 1)
input_padded shape = (1, 64, 2002, 2002)
cols shape before concatenation = (1, 576, 4000000)
cols shape = (576, 4000000)


In [ ]:
# print('d[:20] =', d[:20])
# print('i[:10, :40] =', i[:10, :40])
# i[:10, :20]
i[:10, 33998:34002]

In [39]:
j[0, 1999:2001]

array([1999,    0])

In [10]:
input_padded = np.pad(input_data, ((0,0), (0,0), (padding, padding), (padding, padding)), mode='constant')

# row i
row_i = i

# column j 
col_j = j

# channel delimiter d
delim_d = d



Trying the `.ravel()` option provided in this [SO thread](https://stackoverflow.com/questions/14386822/fast-numpy-fancy-indexing)

In [11]:
a = np.random.randn(3218, 6)
print(a.shape)

rows = np.random.randint(a.shape[0], size=2000)
cols = np.array([1,3,4,5])

result2 = (a.ravel()[(cols + (rows * a.shape[1]).reshape((-1,1))).ravel()]).reshape(rows.size, cols.size)
print(result2.shape)

(3218, 6)
(2000, 4)


In [16]:
np.ravel?

Signature: np.ravel(a, order='C')
Docstring:
Return a contiguous flattened array.

A 1-D array, containing the elements of the input, is returned.  A copy is
made only if needed.

As of NumPy 1.10, the returned array will have the same type as the input
array. (for example, a masked array will be returned for a masked array
input)

Parameters
----------
a : array_like
    Input array.  The elements in `a` are read in the order specified by
    `order`, and packed as a 1-D array.
order : {'C','F', 'A', 'K'}, optional

    The elements of `a` are read using this index order. 'C' means
    to index the elements in row-major, C-style order,
    with the last axis index changing fastest, back to the first
    axis index changing slowest.  'F' means to index the elements
    in column-major, Fortran-style order, with the
    first index changing fastest, and the last index changing
    slowest. Note that the 'C' and 'F' options take no account of
    the memory layout of the underlying array

Trying the `np.ix_` method from the same [SO thread](https://stackoverflow.com/questions/14386822/fast-numpy-fancy-indexing)

In [17]:
np.ix_?

Signature: np.ix_(*args)
Docstring:
Construct an open mesh from multiple sequences.

This function takes N 1-D sequences and returns N outputs with N
dimensions each, such that the shape is 1 in all but one dimension
and the dimension with the non-unit shape value cycles through all
N dimensions.

Using `ix_` one can quickly construct index arrays that will index
the cross product. ``a[np.ix_([1,3],[2,5])]`` returns the array
``[[a[1,2] a[1,5]], [a[3,2] a[3,5]]]``.

Parameters
----------
args : 1-D sequences
    Each sequence should be of integer or boolean type.
    Boolean sequences will be interpreted as boolean masks for the
    corresponding dimension (equivalent to passing in
    ``np.nonzero(boolean_sequence)``).

Returns
-------
out : tuple of ndarrays
    N arrays with N dimensions each, with N the number of input
    sequences. Together these arrays form an open mesh.

See Also
--------
ogrid, mgrid, meshgrid

Examples
--------
>>> a = np.arange(10).reshape(2, 5)
>>> a
array(

In [22]:
print('i shape =', i.shape)
print('j shape =',j.shape)
print('d shape =',d.shape)
print('input_padded shape =',input_padded.shape)
print('cols shape before concatenation =', input_padded[:, d, i, j].shape)
print('cols shape =', cols.shape) # cols = np.concatenate(cols, axis=-1)

i shape = (576, 4000000)
j shape = (576, 4000000)
d shape = (576, 1)
input_padded shape = (1, 64, 2002, 2002)
cols shape before concatenation = (1, 576, 4000000)
cols shape = (6,)


In [27]:
print(i[:100])

[[   0    0    0 ... 1999 1999 1999]
 [   0    0    0 ... 1999 1999 1999]
 [   0    0    0 ... 1999 1999 1999]
 ...
 [   2    2    2 ... 2001 2001 2001]
 [   2    2    2 ... 2001 2001 2001]
 [   0    0    0 ... 1999 1999 1999]]


In [18]:
a = np.random.randn(1, 64, 2002, 2002)
rows = np.random.randint(a.shape[0], size=2000)
cols = np.random.randint(a.shape[1], size=6)

ix_ = np.ix_(rows, cols)
result1 = a[ix_]

In [21]:
cols.shape

(6,)

In [16]:
np.ix_?

Signature: np.ix_(*args)
Docstring:
Construct an open mesh from multiple sequences.

This function takes N 1-D sequences and returns N outputs with N
dimensions each, such that the shape is 1 in all but one dimension
and the dimension with the non-unit shape value cycles through all
N dimensions.

Using `ix_` one can quickly construct index arrays that will index
the cross product. ``a[np.ix_([1,3],[2,5])]`` returns the array
``[[a[1,2] a[1,5]], [a[3,2] a[3,5]]]``.

Parameters
----------
args : 1-D sequences
    Each sequence should be of integer or boolean type.
    Boolean sequences will be interpreted as boolean masks for the
    corresponding dimension (equivalent to passing in
    ``np.nonzero(boolean_sequence)``).

Returns
-------
out : tuple of ndarrays
    N arrays with N dimensions each, with N the number of input
    sequences. Together these arrays form an open mesh.

See Also
--------
ogrid, mgrid, meshgrid

Examples
--------
>>> a = np.arange(10).reshape(2, 5)
>>> a
array(

### Experimenting with the `np.take` to see if it's faster

In [8]:
m = [1, 7, 12, 40]
r = np.arange(5000)
r = np.delete(r, m, axis=0)

x = np.random.rand(5000,5000,10)

%timeit tmp = x[:,m,:]

%timeit tmp = np.delete(x, r, axis=1)

%timeit tmp = np.take(x, m, axis=1)

149 µs ± 2.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
169 µs ± 976 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
136 µs ± 2.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
input_data=out
weights_dict = weights
prefix = 'layers.1.0.'
padding=1
stride=1

im2col_start = time.perf_counter()
if len(input_data.shape) == 4:
    
    batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
        
elif len(input_data.shape) == 3:

    input_data = input_data.reshape((1, 1, 2000 , 2000))
    batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)

# Padding
input_padded = np.pad(input_data, ((0,0), (0,0), (padding, padding), (padding, padding)), mode='constant')
i, j, d = get_indices(input_data=input_data, weights_dict=weights_dict, prefix=prefix)


get_indices takes: 4.249502338992897 seconds


In [10]:
print('i shape =', i.shape)
print('j shape =',j.shape)
print('d shape =',d.shape)
print('input_padded shape =',input_padded.shape)
print('cols shape before concatenation =', input_padded[:, d, i, j].shape)
print('cols shape =', cols.shape)

i shape = (576, 4000000)
j shape = (576, 4000000)
d shape = (576, 1)
input_padded shape = (1, 64, 2002, 2002)
cols shape before concatenation = (1, 576, 4000000)
cols shape = (576, 4000000)


In [11]:
input_padded.shape

(1, 64, 2002, 2002)

In [12]:
# slice_start = time.perf_counter()
# dcols = np.take(input_padded, d, axis=1)
# icols = np.take(input_padded, i, axis=2)
# jcols = np.take(input_padded, j, axis=3)

# slice_end = time.perf_counter()
# print('Slicing takes', slice_end-slice_start)

### Experimenting with the transpose of the data array to see if its faster - **IT'S NOT**

In [13]:
input_data=out
weights_dict = weights
prefix = 'layers.1.0.'
padding=1
stride=1

im2col_start = time.perf_counter()
if len(input_data.shape) == 4:
    
    batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
        
elif len(input_data.shape) == 3:

    input_data = input_data.reshape((1, 1, 2000 , 2000))
    batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)

# Padding
input_padded = np.pad(input_data, ((0,0), (0,0), (padding, padding), (padding, padding)), mode='constant')
i, j, d = get_indices(input_data=input_data, weights_dict=weights_dict, prefix=prefix)
# Multi-dimensional arrays indexing.
input_padded = input_padded.T
cols = input_padded[j, i, d, :].T
cols = np.concatenate(cols, axis=-1)

im2col_end = time.perf_counter()
print('Total Im2col takes:', im2col_end-im2col_start, 'seconds')

get_indices takes: 4.482630530001188 seconds
Total Im2col takes: 15.671476784998958 seconds


In [14]:
print('i shape =', i.shape)
print('j shape =',j.shape)
print('d shape =',d.shape)
print('input_padded shape =',input_padded.shape)
print('cols shape before concatenation =', input_padded[:, d, i, j].shape)
print('cols shape =', cols.shape)

i shape = (576, 4000000)
j shape = (576, 4000000)
d shape = (576, 1)
input_padded shape = (2002, 2002, 64, 1)


IndexError: index 64 is out of bounds for axis 2 with size 64

In [ ]:
i, j, d = get_indices(input_data=input_data, weights_dict=weights_dict, prefix=prefix)


In [ ]:
im2col_testing(input_data=out, weights_dict=weights, prefix='layers.1.0.');

In [ ]:
last = np_Conv2d(input_data=out2, weights_dict=weights, prefix='layers.18.0.')